# Dask clusters

(the material is based on the notebook https://github.com/jrbourbeau/hacking-dask)

This notebook covers Dask's distributed clusters in detail.

## Cluster overview

In this section we'll discuss:

1. The different components which make up a Dask cluster
2. Survey different ways to launch a cluster

![dask schedulers](img/dask-cluster.svg)

### Components of a cluster

A Dask cluster is composed of three different types of objects:

1. **Scheduler**: A single, centralized scheduler process which responds to requests for computations, maintains relavant state about tasks and worker, and sends tasks to workers to be computed.
2. **Workers**: One or more worker processes which compute tasks and store/serve their results.
3. **Clients**: One or more client objects which are the user-facing entry point to interact with the cluster.

A couple of notes about workers:

- Each worker runs in its own Python process. Each worker Python process has its own `concurrent.futures.ThreadPoolExecutor` which is uses to compute tasks in parallel.
- There's actually a fourth cluster object which is often not discussed: the **Nanny**. By default Dask workers are launched and managed by a separate nanny process. This separate process allows workers to restart themselves if you want to use the `Client.restart` method, or to restart workers automatically if they get above a certain memory limit threshold.

#### Related Documentation

- [Cluster architecture](https://distributed.dask.org/en/latest/#architecture)
- [Journey of a task](https://distributed.dask.org/en/latest/journey.html)

## Deploying Dask clusters

Deploying a Dask cluster means launching scheduler, worker, and client processes and setting up the appropriate network connections so these processes can communicate with one another. Dask clusters can be lauched in a few different ways which we will discuss later.

### Manual setup

Launch a scheduler process using the `dask-scheduler` command line utility:

```terminal
$ dask-scheduler
2023-07-20 20:42:29,894 - distributed.scheduler - INFO - -----------------------------------------------
2023-07-20 20:42:30,432 - distributed.scheduler - INFO - State start
2023-07-20 20:42:30,438 - distributed.scheduler - INFO - -----------------------------------------------
2023-07-20 20:42:30,439 - distributed.scheduler - INFO - Clear task state
2023-07-20 20:42:30,440 - distributed.scheduler - INFO -   Scheduler at: tcp://192.168.150.50:8786
2023-07-20 20:42:30,440 - distributed.scheduler - INFO -   dashboard at:                     :8787
2023-07-20 20:42:31,810 - distributed.scheduler - INFO - Register worker <WorkerState 'tls://oksana-2eshadura-40cern-2ech.dask-worker.coffea-opendata.casa:8788', name: kubernetes-worker-50d255ef-ba72-4162-af66-23fdbf942bb3, status: undefined, memory: 0, processing: 0>
2023-07-20 20:42:31,813 - distributed.scheduler - INFO - Starting worker compute stream, tls://oksana-2eshadura-40cern-2ech.dask-worker.coffea-opendata.casa:8788
2023-07-20 20:42:31,813 - distributed.core - INFO - Starting established connection
```

and then launch several workers by using the `dask-worker` command and providing them the address of the scheduler they should connect to:

```terminal
$ dask-worker tcp://192.0.0.100:8786
2023-07-20 20:45:36,940 - distributed.worker - INFO -       Start worker at: tcp://192.168.150.50:38077
2023-07-20 20:45:36,941 - distributed.worker - INFO -          Listening to: tcp://192.168.150.50:38077
2023-07-20 20:45:36,941 - distributed.worker - INFO -          dashboard at:       192.168.150.50:34591
2023-07-20 20:45:36,941 - distributed.worker - INFO - Waiting to connect to:  tcp://192.168.150.50:8786
2023-07-20 20:45:36,941 - distributed.worker - INFO - -------------------------------------------------
2023-07-20 20:45:36,941 - distributed.worker - INFO -               Threads:                        112
2023-07-20 20:45:36,941 - distributed.worker - INFO -                Memory:                   4.00 GiB
2023-07-20 20:45:36,941 - distributed.worker - INFO -       Local Directory: /home/cms-jovyan/dask-worker-space/worker-q_4j0zc5
2023-07-20 20:45:36,941 - distributed.worker - INFO - -------------------------------------------------
2023-07-20 20:45:36,949 - distributed.worker - INFO -         Registered to:  tcp://192.168.150.50:8786
2023-07-20 20:45:36,949 - distributed.worker - INFO - -------------------------------------------------
2023-07-20 20:45:36,949 - distributed.core - INFO - Starting established connection
                         
```

## Cluster managers 

Dask has the notion of cluster manager objects. Cluster managers offer a consistent interface for common activities like adding/removing workers to a cluster, retrieving logs, etc.

![dask schedulers](img/dask-cluster-manager.svg)

### Dask LocalCluster

LocalCluster creates a "cluster" of a scheduler and workers running on the local machine.

In [10]:
from dask.distributed import LocalCluster
# Launch a scheduler and 4 workers on my local machine
cluster = LocalCluster(n_workers=2, threads_per_worker=2)
cluster

LocalCluster(77c84e16, 'tcp://127.0.0.1:34235', workers=2, threads=4, memory=4.00 GiB)

In [11]:
# Scale up to 10 workers
cluster.scale(3)

LocalCluster(77c84e16, 'tcp://127.0.0.1:34235', workers=2, threads=4, memory=4.00 GiB)

In [12]:
# Scale down to 2 workers
cluster.scale(2)

In [8]:
# Retrieve cluster logs
cluster.get_logs()

2023-07-24 03:48:18,893 - distributed.nanny - WARNING - Worker process still alive after 3.1999986267089846 seconds, killing
2023-07-24 03:48:18,895 - distributed.nanny - WARNING - Worker process still alive after 3.1999990844726565 seconds, killing
2023-07-24 03:48:18,987 - distributed.nanny - WARNING - Worker process still alive after 3.1999992370605472 seconds, killing
2023-07-24 03:48:18,989 - distributed.nanny - WARNING - Worker process still alive after 3.1999990844726565 seconds, killing
2023-07-24 03:48:18,991 - distributed.nanny - WARNING - Worker process still alive after 3.1999990844726565 seconds, killing
2023-07-24 03:48:19,087 - distributed.nanny - WARNING - Worker process still alive after 3.1999992370605472 seconds, killing
2023-07-24 03:48:19,090 - distributed.nanny - WARNING - Worker process still alive after 3.199999389648438 seconds, killing
2023-07-24 03:48:19,185 - distributed.nanny - WARNING - Worker process still alive after 3.199999389648438 seconds, killing


{'Cluster': '',
 'Scheduler': "2023-07-24 03:48:14,787 - distributed.scheduler - INFO - Remove worker <WorkerState 'tcp://127.0.0.1:38725', name: 16, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1690170494.7878122')\n2023-07-24 03:48:14,787 - distributed.scheduler - INFO - Remove worker <WorkerState 'tcp://127.0.0.1:35451', name: 15, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1690170494.7871118')\n2023-07-24 03:48:14,786 - distributed.scheduler - INFO - Remove worker <WorkerState 'tcp://127.0.0.1:42017', name: 14, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1690170494.7863724')\n2023-07-24 03:48:14,785 - distributed.scheduler - INFO - Remove worker <WorkerState 'tcp://127.0.0.1:33717', name: 13, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1690170494.7854922')\n2023-07-24 03:48:14,692 - distributed.scheduler - INFO - Remove worker <WorkerState 'tcp://127.0.0.1:41357', name: 11, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1690170494.6922975')\n2023-07-24 03:48:14,691 - distributed.scheduler - INFO - Remove worker <WorkerState 'tcp://127.0.0.1:34563', name: 12, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1690170494.691738')\n2023-07-24 03:48:14,691 - distributed.scheduler - INFO - Remove worker <WorkerState 'tcp://127.0.0.1:39327', name: 9, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1690170494.6910913')\n2023-07-24 03:48:14,690 - distributed.scheduler - INFO - Remove worker <WorkerState 'tcp://127.0.0.1:45229', name: 10, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1690170494.6898422')\n2023-07-24 03:48:14,412 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:35451\n2023-07-24 03:48:14,412 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:35451', name: 15, status: closed, memory: 0, processing: 0>\n2023-07-24 03:48:14,411 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:33717\n2023-07-24 03:48:14,410 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:33717', name: 13, status: closed, memory: 0, processing: 0>\n2023-07-24 03:48:14,390 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:41357\n2023-07-24 03:48:14,388 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:41357', name: 11, status: closed, memory: 0, processing: 0>\n2023-07-24 03:48:14,387 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:42017\n2023-07-24 03:48:14,386 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:42017', name: 14, status: closed, memory: 0, processing: 0>\n2023-07-24 03:48:13,691 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:38725\n2023-07-24 03:48:13,690 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:38725', name: 16, status: closed, memory: 0, processing: 0>\n2023-07-24 03:48:13,688 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:45229\n2023-07-24 03:48:13,687 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:45229', name: 10, status: closed, memory: 0, processing: 0>\n2023-07-24 03:48:12,492 - distributed.scheduler - INFO - Starting worker compute stream, tcp://127.0.0.1:34563\n2023-07-24 03:48:12,490 - distributed.scheduler - INFO - Register worker <WorkerState 'tcp://127.0.0.1:34563', name: 12, status: closed, memory: 0, processing: 0>\n2023-07-24 03:47:31,787 - distributed.scheduler - INFO - Remove worker <WorkerState 'tcp://127.0.0.1:46411', name: 8, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1690170451.7875643')\n2023-07-24 03:47:31,696 - distributed.scheduler - INFO - Rem

In [9]:
# Shut down cluster
cluster.close()

There are several projects in the Dask ecosystem for easily deploying clusters on commonly used computing resources:

- [Dask-Jobqueue](https://jobqueue.dask.org/en/latest/) for deploying Dask on job queuing systems (e.g. PBS, Slurm, etc.)
- [Dask-Kubernetes](https://kubernetes.dask.org/en/latest/) for deploying Dask using native Kubernetes APIs
- [Dask-Cloudprovider](https://cloudprovider.dask.org/en/latest/) for deploying Dask clusters on various cloud platforms (e.g. AWS, GCP, Azure, etc.)
- [Dask-Yarn](https://yarn.dask.org/en/latest/) for deploying Dask on YARN clusters
- [Dask-MPI](http://mpi.dask.org/en/latest/) for deploying Dask on existing MPI environments

Launching clusters with any of these projects follows a similar pattern as using Dask's built-in `LocalCluster`:

```python

# Launch a Dask cluster on a HTCondor job queueing system [For this you will need HTCondor related configurations]
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(...)


# Launch a Dask cluster on a SLURM job queueing system [For this you will need SLURM related configurations]
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(...)


# Launch a Dask cluster on a PBS job queueing system [For this you will need PBS related configurations]
from dask_jobqueue import PBSCluster
cluster = PBSCluster(...)

# Launch a Dask cluster on a Kubernetes cluster [For this you will need Kubernetes related configurations]
from dask_kubernetes import KubeCluster
cluster = KubeCluster(...)

```

#### Related Documentation

- [Cluster setup](https://docs.dask.org/en/latest/setup.html)